### Imports

In [1]:
from main import QuantumIR
from qiskit_aer import AerSimulator
from qiskit import QuantumCircuit
import numpy as np

### Generate the IR

In [ ]:
# Run
quantum_ir = QuantumIR()
quantum_ir.run_dataclass()
quantum_ir.run_generate_ir()
quantum_ir.run_transformations()

### Transform CCNOT gates

In [ ]:
quantum_ir.metrics_transformation()
#quantum_ir.run_transformations()

### Find input and output number

In [ ]:
module = quantum_ir.module
funcOp = module.body.block._first_op

# Numbers and type of the input parameters
input_args = funcOp.body.block._args
input_number = input_args.__len__()
input_types = [arg.type for arg in input_args]

# First and last op
first_op = funcOp.body.block._first_op
last_op = funcOp.body.block._last_op
output_number = 0
init_number = 0

current = first_op
while(current is not None):
    if current.name == "quantum.init":
        init_number += 1
    if current.name == "quantum.measure":
        output_number += 1
    current = current.next_op

qubit_number = input_number + init_number

print("Number of inputs: ", input_number, "\nNumber of support qubits: ", init_number, "\nTotal qubit used: ", qubit_number, "\nNumber of outputs: ", output_number)

### Initialize circuit

In [ ]:

def create_circuit(first_op, input_number, output_number, state):

    circuit = QuantumCircuit(input_number, output_number)
    current = first_op
    cbit_index = 0

    qubit_list = [i for i in range(input_number)]

    circuit.initialize(state, qubit_list)

    while(current is not None):
        # find the indexes of the qubit
        operands_names = [op._name for op in current.operands]
        indexes = [int(name.split("_")[0][1]) for name in operands_names]
        if current.name == "quantum.not":
            circuit.x(indexes[0])
        if current.name  == "quantum.cnot":
            circuit.cx(indexes[0], indexes[1])
        if current.name == "quantum.ccnot":
            circuit.ccx(indexes[0], indexes[1], indexes[2])
        if current.name == "quantum.h":
            circuit.h(indexes[0])
        if current.name == "quantum.t":
            circuit.t(indexes[0])
        if current.name == "quantum.tdagger":
            circuit.tdg(indexes[0])
        if current.name == "quantum.measure":
            circuit.measure(indexes[0], cbit_index)
            cbit_index += 1
        
        current = current.next_op
    
    return circuit

# initializing it to an example state (e.g. all qubit at 0)
example_state = np.array([1] + [0] * (2**qubit_number - 1))
create_circuit(first_op, qubit_number, output_number, example_state).draw(output='mpl')



### Truth tables

In [ ]:
from numpy import eye

istates = eye(2**qubit_number)

def bit_strings_iterative(N):
    bit_list = ['']
    for _ in range(N):
        bit_list = ['0' + bit_string for bit_string in bit_list] + \
                   ['1' + bit_string for bit_string in bit_list]
    return bit_list

# tutte le stringhe di bit di qubit_number bit
istr = bit_strings_iterative(qubit_number)

print(istr)
print(istates)

In [ ]:
from qiskit.providers import JobStatus

counts = {}
quantum_truth_table = {}
backend = AerSimulator()
qubit_list = [i for i in range(qubit_number)]

for i in range(2**qubit_number):
    circuit = create_circuit(first_op, qubit_number, output_number, istates[i])

    job = backend.run(circuit, shots=2000)
    result = job.result()

    # Consider only the entries where the support output qubits are set to zero.
    if istr[i][:init_number] == '0' * init_number:
        counts[istr[i][init_number:]] = result.get_counts()

    for outer_key, inner_dict in counts.items():
        for inner_key, value in inner_dict.items():
            quantum_truth_table[outer_key] = inner_key

print(quantum_truth_table)


Notice that, both for input and output qubits, the q[0] is the rightmost one (little endian).

In [ ]:
import csv

# Initialize the dictionary
classical_truth_table = {}

# Read the CSV file
with open('truth-tables/xorInPlace.csv', 'r') as file:
    csv_reader = csv.DictReader(file)
    
    # Dynamically find "out" columns
    out_columns = [col for col in csv_reader.fieldnames if col.startswith('out')]
    
    for row in csv_reader:
        # Extract the values of "out" columns, reverse them, and join as a string
        flipped_value = ''.join(str(row[col]) for col in reversed(out_columns))
        classical_truth_table[row['Inputs']] = flipped_value

print(classical_truth_table)

In [ ]:
equal = quantum_truth_table == classical_truth_table

# Print the result
print("Are the truth tables equal?", equal)

if not equal:
    for key in quantum_truth_table:
        if quantum_truth_table[key] != classical_truth_table.get(key):
            print(f"Difference found at key {key}: new_counts has {quantum_truth_table[key]}, data_dict has {classical_truth_table.get(key)}")

# Metriche
Circuit depth, circuit width (only number of qubits, qiskit counts also classical bits), gate count, t gate count, t gate depth.

In [ ]:
print("Depth of the quantum circuit:", circuit.depth())
print("Number of qubits in the quantum circuit:", circuit.num_qubits)
print("Number of gates in the quantum circuit:", circuit.size())

#print("T gate count for the circuit ",circuit.count_ops()['t'] + circuit.count_ops()['tdg'])
#print("T gate depth: ", circuit.depth(lambda gate: gate.operation.name in ['t', 'tdg']))

In [ ]:
from qiskit.converters import circuit_to_dag
from qiskit.dagcircuit.dagcircuit import DAGOpNode

# Step 1: Convert to a DAGCircuit
dag = circuit_to_dag(circuit)
tgatewidth=0
# Step 2: Retrieve the critical path
# Each element in the critical path is a node in the DAG that contributes to this longest path
critical_path = dag.longest_path()
print("Critical Path:")
for node in critical_path:
    if isinstance(node, DAGOpNode): 
        qubit_indices = [circuit.qubits.index(q) for q in node.qargs] 
        #print(f"{node.name} {tuple(qubit_indices)}")
        if(node.name=="tdg" or node.name=="t"):
            tgatewidth+=1
print("Critical Path Length:", len(critical_path))
print("T Gate Width:", tgatewidth)